<a href="https://colab.research.google.com/github/Ananassio/Big-Data-Analytics/blob/main/week_5/Assigment_NY_Taxi_on_Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment: NY Taxi Fare Prediction with DASK

* Note: see [NY Taxi Usecase Notebook](https://colab.research.google.com/github/keuperj/DataEngineering22/blob/main/week_8/UseCase_NY_Taxi.ipynb) for an discription of a non-parallel solution

* NY Taxi Fare Prediction Task + Data: https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/overview

In [1]:
#install DASK
!pip install distributed "dask[complete]" dask-ml graphviz  --ignore-installed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 837 kB 5.5 MB/s 
     |████████████████████████████████| 1.1 MB 57.3 MB/s 
     |████████████████████████████████| 148 kB 18.6 MB/s 
     |████████████████████████████████| 46 kB 3.2 MB/s 
     |████████████████████████████████| 596 kB 50.2 MB/s 
     |████████████████████████████████| 133 kB 36.5 MB/s 
     |████████████████████████████████| 428 kB 55.7 MB/s 
     |████████████████████████████████| 299 kB 15.8 MB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 1.2 MB 52.4 MB/s 
     |████████████████████████████████| 281 kB 68.2 MB/s 
     |████████████████████████████████| 40 kB 4.6 MB/s 
     |████████████████████████████████| 96 kB 4.3 MB/s 
     |████████████████████████████████| 140 kB 43.7 MB/s 
     |████████████████████████████████| 98 kB 6.9 MB/s 
     |████████████████████████████████| 38

In [1]:
#in colab, we need to clone the data from the repo
!git clone https://github.com/keuperj/DATA.git
path='DATA'

Cloning into 'DATA'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 101 (delta 2), reused 14 (delta 2), pack-reused 87
Receiving objects: 100% (101/101), 146.44 MiB | 26.71 MiB/s, done.
Resolving deltas: 100% (23/23), done.
Checking out files: 100% (69/69), done.


In [98]:
#read data to Pandas DF
import pandas as pd
data=pd.read_csv("DATA/NY_taxi_train_small.csv")
y=data['fare_amount']
X=data.drop(['fare_amount'],axis=1)


In [99]:
X.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2014-03-12 17:04:30.0000001,2014-03-12 17:04:30 UTC,-73.956721,40.767081,-73.989080,40.772745,1
1,2009-04-17 21:59:00.00000044,2009-04-17 21:59:00 UTC,-73.870913,40.773722,-73.996285,40.716018,2
2,2009-10-06 13:42:00.00000015,2009-10-06 13:42:00 UTC,-73.976258,40.751410,-73.984795,40.751305,3
3,2012-05-02 21:38:39.0000004,2012-05-02 21:38:39 UTC,-73.977940,40.752586,-73.976525,40.667005,1
4,2011-04-21 18:11:13.0000001,2011-04-21 18:11:13 UTC,-73.988390,40.723152,-73.984367,40.736301,1


In [100]:
y.head()

0    11.0
1    23.7
2     5.7
3    28.1
4     5.7
Name: fare_amount, dtype: float64

## Assignment:
use *DASK Dataframes* and *DASK-ML* to 
* split in train and test data
* build a pre-processing,
* feature extraction and 
* prediction pipeline 

to predict the taxi fares. 

### Use
* local DASK cluster
* [DASK DataFrames](https://examples.dask.org/dataframe.html)
* [DASK-ML](https://ml.dask.org/)

### Hints:
* start with a very simple, but working predicten and then enhance the solution with better pre-processing and features

In [122]:
# Cluster setup
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

/usr/local/lib/python3.7/dist-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37247 instead
  f"Port {expected} is already in use.\n"


In [123]:
#get client info
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:37247/status,
Dashboard: http://127.0.0.1:37247/status,Workers: 2
Total threads: 2,Total memory: 12.68 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35937,Workers: 2
Dashboard: http://127.0.0.1:37247/status,Total threads: 2
Started: Just now,Total memory: 12.68 GiB
Comm: tcp://127.0.0.1:33131,Total threads: 1
Dashboard: http://127.0.0.1:38919/status,Memory: 6.34 GiB
Nanny: tcp://127.0.0.1:34923,


In [113]:
import dask
import dask.dataframe as dd
import dask.array as da
import numpy as np

### Export DASk Dashboard to public URL

In [126]:
!npm install -g npm

/tools/node/bin/npm -> /tools/node/lib/node_modules/npm/bin/npm-cli.js
/tools/node/bin/npx -> /tools/node/lib/node_modules/npm/bin/npx-cli.js
+ npm@8.13.0
added 63 packages from 18 contributors, removed 296 packages and updated 138 packages in 10.966s


In [127]:
!npm install -g localtunnel


changed 22 packages, and audited 23 packages in 1s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities


In [128]:
!lt --port 8787


^C


In [101]:
Xd = dask.dataframe.from_pandas(X.drop(['pickup_datetime','key'],axis=1), npartitions=10)
Xd

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
npartitions=10,,,,,
0,float64,float64,float64,float64,int64
100000,...,...,...,...,...
...,...,...,...,...,...
900000,...,...,...,...,...
999999,...,...,...,...,...


In [102]:
Xd.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,-73.956721,40.767081,-73.989080,40.772745,1
1,-73.870913,40.773722,-73.996285,40.716018,2
2,-73.976258,40.751410,-73.984795,40.751305,3
3,-73.977940,40.752586,-73.976525,40.667005,1
4,-73.988390,40.723152,-73.984367,40.736301,1


In [103]:
from dask_ml.preprocessing import StandardScaler
Xd = StandardScaler().fit_transform(Xd)
Xd.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,-0.116695,0.087540,-0.122419,0.092280,-0.516770
1,-0.109739,0.088228,-0.123019,0.086211,0.238363
2,-0.118279,0.085918,-0.122062,0.089986,0.993496
3,-0.118415,0.086040,-0.121372,0.080969,-0.516770
4,-0.119262,0.082993,-0.122026,0.088381,-0.516770


In [115]:
Xd.replace([np.inf, -np.inf], np.nan)
Xd = Xd.fillna(0)

Xd.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,-0.116695,0.087540,-0.122419,0.092280,-0.516770
1,-0.109739,0.088228,-0.123019,0.086211,0.238363
2,-0.118279,0.085918,-0.122062,0.089986,0.993496
3,-0.118415,0.086040,-0.121372,0.080969,-0.516770
4,-0.119262,0.082993,-0.122026,0.088381,-0.516770


In [105]:
Xd.astype(float)

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
npartitions=10,,,,,
0,float64,float64,float64,float64,float64
100000,...,...,...,...,...
...,...,...,...,...,...
900000,...,...,...,...,...
999999,...,...,...,...,...


In [106]:
yd = dask.dataframe.from_pandas(y, npartitions=10)
yd

Dask Series Structure:
npartitions=10
0         float64
100000        ...
           ...   
900000        ...
999999        ...
Name: fare_amount, dtype: float64
Dask Name: from_pandas, 10 tasks

In [107]:
yd.head()

0    11.0
1    23.7
2     5.7
3    28.1
4     5.7
Name: fare_amount, dtype: float64

In [108]:
yd.fillna(0)

Dask Series Structure:
npartitions=10
0         float64
100000        ...
           ...   
900000        ...
999999        ...
Name: fare_amount, dtype: float64
Dask Name: fillna, 20 tasks

In [116]:
from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xd, yd, test_size=0.15)

/usr/local/lib/python3.7/dist-packages/dask_ml/model_selection/_split.py:469: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  category=FutureWarning,


In [110]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
import joblib

In [81]:
RFR = RandomForestRegressor()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,-0.116695,0.087540,-0.122419,0.092280,-0.516770
1,-0.109739,0.088228,-0.123019,0.086211,0.238363
2,-0.118279,0.085918,-0.122062,0.089986,0.993496
3,-0.118415,0.086040,-0.121372,0.080969,-0.516770
4,-0.119262,0.082993,-0.122026,0.088381,-0.516770


In [118]:
%%time
with joblib.parallel_backend('dask'):
    # Your scikit-learn code
    model = RFR.fit(X_train.head(1000), y_train.head(1000)) 

CPU times: user 893 ms, sys: 148 ms, total: 1.04 s
Wall time: 11.4 s


In [120]:
model.score(X_test.head(100), y_test.head(100))

0.5469747905322506